## Import

In [1]:
import q_learner
import state_string_utils as stringutils
import state_enumerator as stateenum
import NAS 
import netparser
from tensorflow import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from datetime import datetime
import time
import importlib
import os

## Loading Data

In [2]:
feat_window = 90

# performance measures window: number of years
pm_window = 3
lb_window = int(3 * pm_window * 365.25) + 1

# Window length between training samples: number of days
sample_window = 30

# test period start
test_start_date = '2020-06-30'

# filepath
filepath = '/Users/kz_ke/Documents/Masters/Classes/DL/WM-SecuritySelection-main/data/MF_LargeCap_ExcessReturn_3Y.parquet'

In [3]:
def prepare_data_for_er_ari(filepath, label_window=pm_window):
    er_ari_df = pd.read_parquet(filepath)
    

    data_dict = {ticker: er_ari_df[ticker].dropna() for ticker in er_ari_df.columns}
    tickers_to_remove = []
    
    label_dict = {}
    for ticker, series in tqdm(data_dict.items()):
        if series.isna().sum() == series.shape[0]:
            tickers_to_remove += [ticker]
            continue

        last_date = series.index[-1] - relativedelta(years=pm_window)
        if last_date <= series.index[0]:
            tickers_to_remove.append(ticker)
            continue

        index = series.loc[:series.index[-1] - relativedelta(years=pm_window)].index
        label_dict[ticker] = pd.Series([
            series[date + relativedelta(years=pm_window)] for date in index
        ], index=index)
        
    _ = [data_dict.pop(ticker) for ticker in tickers_to_remove]
    
    return data_dict, label_dict

In [4]:
%%time
data_dict, label_dict = prepare_data_for_er_ari(filepath)

100%|██████████████████████████████████████████████████████████████████████████████| 1330/1330 [04:45<00:00,  4.66it/s]

Wall time: 4min 48s


In [5]:
%%time

tickers = list(data_dict.keys())

train_data = []
train_labels = []

test_data = []
test_labels = []

# test start date
checkpoint = datetime.strptime(test_start_date, '%Y-%m-%d') - relativedelta(years=pm_window)

for ticker in tqdm(tickers):    
    label = label_dict[ticker]
    if label.shape[0] == 0:
        continue
    ts = data_dict[ticker].loc[:label.index[-1]]

    indices = [np.arange(i, i+lb_window, feat_window) for i in range(0, ts.shape[0] - lb_window + 1, sample_window)]
    
    temp_data = np.array([ts.iloc[sub_indices].values for sub_indices in indices])
    if temp_data.shape[0] == 0:
        continue
    temp_labels = np.array([label.loc[ts.index[sub_indices[-1]]] for sub_indices in indices])
    
    train_indices = [idx for idx in range(temp_data.shape[0]) if ts.index[indices[idx][-1]] <= checkpoint]
    test_indices = [idx for idx in range(temp_data.shape[0]) if ts.index[indices[idx][-1]] > checkpoint]
    
    train_data += [temp_data[train_indices]] 
    train_labels += [temp_labels[train_indices]]
    
    test_data += [temp_data[test_indices]] 
    test_labels += [temp_labels[test_indices]]



100%|█████████████████████████████████████████████████████████████████████████████| 1130/1130 [00:09<00:00, 122.02it/s]

Wall time: 9.27 s


In [6]:
def reshape_input_data(x=None, y=None):
    if x is not None:
        if len(x.shape) == 2:
            x = x.reshape(-1, 1, x.shape[1])
        elif len(x.shape) == 3:
            x = x.reshape(-1, x.shape[2], x.shape[1])
        else:
            raise ValueError('Invalid x shape: {}'.format(x.shape))

    if y is not None:
        if len(y.shape) == 1:
            y = y.reshape(-1, 1)
        elif len(y.shape) == 2:
            pass
        else:
            raise ValueError('Invalid y shape: {}'.format(y.shape))

    if x is None and y is None:
        return None
    elif x is None and y is not None:
        return y
    elif x is not None and y is None:
        return x
    else:
        return x, y

In [7]:
%%time
x_train = np.concatenate(train_data)[:, :, np.newaxis]
y_train = np.concatenate(train_labels)[:, np.newaxis]

x_test = np.concatenate(test_data)[:, :, np.newaxis]
y_test = np.concatenate(test_labels)

Wall time: 14.7 ms


In [8]:
train_len = int(len(x_train) * 0.9)
x_train, x_val = x_train[:train_len], x_train[train_len:]
y_train, y_val = y_train[:train_len], y_train[train_len:]

In [9]:
x_train.shape

(42346, 37, 1)

In [10]:
x_val.shape

(4706, 37, 1)

## Running NAS

In [11]:
path = '/Users/kz_ke/Documents/Masters/Classes/DL/AutoML3/mylogs'
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
    print("The new directory is created!")

In [12]:
_model = __import__('models.AutoML',
                    globals(),
                    locals(),
                    ['state_space_parameters', 'hyper_parameters'], 
                    0)

In [18]:
epsilon=0.9

In [20]:
factory = NAS.NAS('mylogs',
                  _model.state_space_parameters,
                  _model.hyper_parameters,
                  epsilon,
                  0.5)

Found replay dictionary
Found q values


In [ ]:
for runtimes in range(73):
    net, i = factory.generate_new_netork()
    print(net)
    p= netparser.parse('net', net)
    newnet = netparser.parse_network_structure(p)
    model = keras.Sequential(newnet)
    
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights = True)
    model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                loss='mean_squared_error',
                metrics=[keras.metrics.RootMeanSquaredError()]
            )
    
    history = model.fit(x_train, y_train, batch_size = 40, epochs = 100, callbacks=[callback], validation_data=(x_val, y_val))
    
    bestval = model.evaluate(x_val, y_val)[1]
    print(model.evaluate(x_test, y_test)[1])
    
    factory.incorporate_trained_net(net, bestval, epsilon, [i])

[GRU(30,leaky_relu), GRU(10,tanh), GRU(20,sigmoid), TERMINATE]
Epoch 1/100
1059/1059 [==============================] - 87s 79ms/step - loss: 0.0012 - root_mean_squared_error: 0.0350 - val_loss: 0.0010 - val_root_mean_squared_error: 0.0321
Epoch 2/100
1059/1059 [==============================] - 90s 85ms/step - loss: 0.0012 - root_mean_squared_error: 0.0341 - val_loss: 9.7540e-04 - val_root_mean_squared_error: 0.0312
Epoch 3/100
1059/1059 [==============================] - 87s 82ms/step - loss: 0.0011 - root_mean_squared_error: 0.0339 - val_loss: 9.7382e-04 - val_root_mean_squared_error: 0.0312
Epoch 4/100
1059/1059 [==============================] - 85s 81ms/step - loss: 0.0011 - root_mean_squared_error: 0.0339 - val_loss: 9.7990e-04 - val_root_mean_squared_error: 0.0313
Epoch 5/100
1059/1059 [==============================] - 86s 81ms/step - loss: 0.0011 - root_mean_squared_error: 0.0338 - val_loss: 9.7722e-04 - val_root_mean_squared_error: 0.0313
Epoch 6/100
1059/1059 [=============

In [16]:
for i in range(10000):
    factory.qlearner.sample_replay_for_update()

## Step by Step Running

In [92]:
net, i = factory.generate_new_netork()
print(net)

[RNN(100,sigmoid), GRU(100,tanh), LSTM(200,linear), FC(50, relu), TERMINATE]


In [121]:
p= netparser.parse('net', net)
newnet = netparser.parse_network_structure(p)

In [122]:
newnet

In [128]:
model = keras.Sequential(newnet)

In [213]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights = True)

In [129]:
model.compile(
            optimizer=keras.optimizers.Adam(lr=1e-4),
            loss='mean_squared_error',
            metrics=[keras.metrics.RootMeanSquaredError()]
        )

In [214]:
model.fit(x_train, y_train, batch_size = 40, epochs =5, callbacks=[callback], validation_data=(x_val, y_val))

1177/1177 [==============================] - 244s 208ms/step - loss: 0.0011 - root_mean_squared_error: 0.0328 - val_loss: 0.0049 - val_root_mean_squared_error: 0.0698


In [182]:
bestval = model.evaluate(x_test, y_test)[1]

328/328 [==============================] - 14s 43ms/step - loss: 0.0045 - root_mean_squared_error: 0.0672


In [61]:
factory.incorporate_trained_net(net, bestval, 1, [1])

Incorporated net, acc: 0.069071, net: [GRU(100,linear), BILSTM(150,relu), FC(50, leaky_relu), FC(1, leaky_relu)]


In [62]:
factory.replay_dictionary

,net,accuracy_best_val,epsilon,iteration
0,"[GRU(100,linear), BILSTM(150,relu), FC(50, lea...",0.069071,1,1
